In [126]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [127]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [128]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [129]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [130]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [131]:
use_case_list = [

"Flashcard.txt",
"WordSortingTest.txt",
"Review test.txt",
]


# "Learning days history.txt",
# "Handbook.txt",
# "EditTest.txt",
# "AddQuestion .txt",
# "AddLesson.txt",
# "Add new topic.txt",
# "Registry.txt",
# "Login.txt",

In [132]:
BASELINE_GENERATE_TC = """
read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form."""

In [133]:
use_case_directory_path = r"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite"
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\baseline 2\matcha\ts"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline 2\matcha\tc 2"

In [134]:

# for usecase in use_case_list:
#   base_name = os.path.basename(usecase)
#   usecase_name = os.path.splitext(base_name)[0]
#   usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
#   testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
#   testscenario = read_file_content(testscenario_path)
#   content = "Use case:" + read_file_content(usecase_path) + "\nTest scenarios:\n"+ testscenario
#   promptExtractCondition = [
#     { "role": "system", "content": BASELINE_GENERATE_TC},
#     { "role": "user", "content": content}
#   ]
#   gpt_response = ask(promptExtractCondition, client, model)
#   print(content)
#   print(gpt_response)
#   print()
#   testcase_path = os.path.join (save_testcase_path,f"{usecase_name}.txt" )
#   write_string_to_file(testcase_path, gpt_response)

In [135]:

for usecase in use_case_list:
  base_name = os.path.basename(usecase)
  usecase_name = os.path.splitext(base_name)[0]
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  print(testscenario_path)
  testscenario_list = read_file_to_list(testscenario_path)

  for testscenario in testscenario_list:
    content = "Test scenarios:\n"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    promptExtractCondition = [
      { "role": "system", "content": BASELINE_GENERATE_TC},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

D:\Dissertation-GPT\evaluate\baseline 2\matcha\ts\Flashcard.txt
{
  "Test Case 1": {
    "testCaseName": "Navigating and Studying Flashcards",
    "objective": "Verify that the learner can navigate through lessons, select a topic, study flashcards, and mark the topic as completed.",
    "testSteps": [
      "Step 1: Log into the application.",
      "Step 2: Click the 'Learn New Word' tab on the navigation bar.",
      "Step 3: Select a lesson from the displayed list.",
      "Step 4: Choose a topic to study from the lesson details.",
      "Step 5: Study the flashcards by flipping them and navigating using the 'Next' and 'Previous' buttons.",
      "Step 6: After studying all flashcards, click the 'Next' button on the last flashcard to be redirected to the 'completion notification' page.",
      "Step 7: Click 'return to the lesson page' on the 'completion notification' page."
    ],
    "expectedResult": "The learner is able to study all flashcards in the topic, and upon completion, 